In [35]:
# 파일 불러오기 
from google.colab import drive 
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [36]:
import os # 필요한 패키지 임포트 

try: 
  os.mkdir(path= "/content/drive/My Drive/colab notebooks/textmining/result/")
except:
  print("already exists")

already exists


In [37]:
# 저장할 경로 설정 
save_dir = "/content/drive/My Drive/colab notebooks/textmining/result/"

In [38]:
# Knolpy 설치 
!pip install konlpy

In [39]:
# 필요한 패키지를 임포트합니다 
import pandas as pd # 데이터프레임 형식 데이터를 다루기 위해 필요합니다 
from konlpy.tag import Okt #twitter 형태소 분석기를 불러옵니다 
import nltk #자연어 처리 프로그램입니다 
import csv #csv 파일을 읽기 위해 필요합니다 
from tqdm import tqdm_notebook as tqdm #진행 상황을 시각화해줍니다 
import datetime as dt # 시간을 계산하기 위해 필요합니다. 

In [40]:
# 진행상황을 텍스트로 표현하기 위한 함수입니다 
def logging(msg):
   a = dt.datetime.strftime(dt.datetime.now(), "%Y-%m-%d %H:%M:%S")
   b = "| " + a.split(" ")[0] + " | " + a.split(" ")[1] + " | " + msg + "\n"
   print(b)

In [76]:
# 감정 분석을 진행하기 위해 미리 수집한 블로그 데이터를 불러옵니다.
blog_data = pd.read_csv("/content/drive/My Drive/colab notebooks/textmining/data/2020_1 final3.csv", encoding = "UTF-8") 
blog_data.head()

,student,structure,effort,evaluation
0,1,"이론 설명이 주가 된다. 하지만, 이론 설명만으로는 한 나라의 문화를 모두 이해하는...","매 수업 전 예습을 해서 오늘 배울 내용이 무엇인지를 먼저 파악했고, 수업이 끝난 ...",프랑스문화권연구 과목을 처음 들어봐서 기존의 대면 수업의 경우 어떤 방식으로 했는지...
1,2,"교수님께서 해당 주차에 들어야 할 강의를 월요일, 수요일 강의로 나누어 차례로 스노...",수업을 제때제때 듣고 수업 자료에 필기를 했다. 수업에 관해 궁금한 것이 생기면 교...,온라인 강의를 통해 전에 들었던 강의를 다시 들을 수 있다는 것은 좋았다. 하지만 ...
2,3,"이론 강의였으므로 수업 전체는 온라인으로 진행되었고, 1주일에 영상과 참고 자료가 ...","도움이 되었던 수업 방법은 직접 자료를 프린트해 필기를 하는 방식이었는데, 단순히 ...",과제 없이 수업이 진행된 점도 도움이 되었던 것 같습니다. 학습 평가 자체는 시험으...
3,4,"이론과 실습이 병행되고 1회에 2시간, 주 2회 진행되는 수업이다. 온라인 수업으로...",수업 시간에 집중해서 들으며 이해한 내용을 최대한 필기로 남겨두려 하였다. 무엇보다...,수업 내용은 전체적으로 흥미롭고 유익했다. 중간중간 지금 배우는 것들이 실생활에서 ...
4,5,온라인으로 수업을 하게 되면서 이해를 못 하거나 궁금한 점이 생길 학생들을 걱정하여...,NaN,온라인 퀴즈와 과제를 실시했기에 기말고사에는 다른과목과 다르게 중간과 기말고사의 범...


In [77]:
"""
감정 분석 결과를 저장할 새로운 열 5개를 데이터셋에 추가합니다.
tokens에는 게시글을 토큰화한 결과를 저장합니다.
pos/nuet/neg 는 게시글의 긍정, 중립, 부정 점수를 저장합니다.
sentiment 는 게시글의 감정 판정 결과를 저장합니다. 
"""

blog_data["tokens"] = None
blog_data["pos"] = None
blog_data["neut"] = None
blog_data["neg"] = None
blog_data["sentiment"] = None

In [78]:
# '#단어: 부정점수: 중립점수: 긍정점수' 형식의 감정 사전을 만듭니다. 
# 사전 이름은 "lexicon" 이라 명명합니다. 

lexicon = dict()

with open('/content/drive/My Drive/colab notebooks/textmining/polarity.csv', 'r', encoding='utf-8-sig') as polarity:
    #next(polarity) # csv 파일에서 첫 행 건너뛰는 용도 
    for line in csv.reader(polarity):
        key = str()
        for word in line[0].split(';'):
            key += word.split('/')[0]

        lexicon[key] = {'Neg': line[3], 'Neut': line[4], 'Pos': line[6]}

In [79]:
# 주어진 텍스트에 대해 토큰화 작업을 하는 함수를 정의합니다. 

def get_token(text):
    pos = []
    tokenizer = Okt() #twitter 형태소 분석기를 불러옵니다.
    if isinstance(text, float) == True: # 주어진 텍스트가 공백일 시 건너뜁니다.
        tokens = "Nothing"
    else:
        tokens = tokenizer.pos(text)
    
    tags = ['Noun', 'Adjective', 'Verb', 'Adverb'] #형태소 중 명사, 형용사, 동사, 부사만 수집합니다. 
    #불용어를 정의합니다.         *불용어: 문장에서 내용을 나타내는데 큰 역할을 하지 않은 기능어. 
    stop_words = ["등", "이", "하", "않", "있", "대하", "하지만", "지", "일", "속", "하", "되", "수업", "및", "교수님", "교수", "강의",
                 "서", "있다", "것", "되었습니다", "하는", "한다", "했다", "주", "하였습니다", "대", "했습니다", "하여", "해보세요",
                  "있는", "입니다", "있음", "되거나", "할", "있는", "관련", "해", "학생","학우님","학우", "없음",
                  "월", "화", "수","목", "금", "토", "일", "를", "나", "그", "위", "고","와", "하며", "의", "가",
                 "하겠습니다", "해야", "몇", "는", "전", "이를","하고", '시킵니다', '된', '있습니다', '제', "이론",
                  "월요일", "화요일", "수요일", "목요일", "금요일", "토요일", "일요일",
                 "있는데요", "있는데", "볼", "로", "은", "총", "우","보였다", "하기", "후", "간", "이번", "때","원",
                 "합시다", "중", "과", "저","분","방","임", "된다","날","함", "있으며","개", "될","며", "보자","합니다","안녕하세요",
                 "같습니다","조","다", "스","가를", "경우", "무엇","대한","내용", "위해", "이후","현재", "보시",
                 "우리", "없는", "모두","언제","어디", "세","사", "미","데", "료", "그린",
                 "한", "꾸준히", "하겠다", "리","두었다", "또한", "들어","통해", "바란다", "다음"," 이어",
                 "됨", "됨", "됌", "또", "됐다", "촬용", "분야", "발표", "매일", "점","출처", "기반", "예정", 
                  "오전", "계획", "오늘", "정부", "더", "마감", "분야", "상","뜻", "특징", "업계", "크게","하나", 
                 "콤", "재", "해서", "같다", "곳", "역시","라며", "처", "첫", "보면", "창", "출", "번", "각각","남기","겸","보기"]
    
    for token in tokens:
        if not tokens == "Nothing":
          if token[1] in tags:
            if not token[0] in stop_words: #불용어는 제외하고 
              pos.append(token[0])  #토큰 수집
        else:
           continue

    return pos

In [80]:
#감성분석 함수 정의
def get_sentiment(token_list):
    cnt = 0 # 평균을 내기 위해 토큰 개수를 셉니다. 
    pos = 0
    neut = 0
    neg = 0
    sentiment = None
    for token in token_list:
        try:
            #lexicon에 토큰이 없으면 건너뜁니다.
            pos += float(lexicon[token]["Pos"]) # 토큰들의 긍정점수를 구해서 더합니다. 
            neut += float(lexicon[token]["Neut"]) # 토큰들의 중립점수를 구해서 더합니다.
            neg += float(lexicon[token]["Neg"]) # 토큰들의 부정점수를 구해서 더합니다.
            cnt += 1
        except:
            continue
            
    if not cnt == 0:   
        pos = pos/cnt # 게시글의 긍정점수 평균을 구합니다. 
        neut = neut/cnt # 게시글의 중립점수 평균을 구합니다. 
        neg = neg/cnt # 게시글의 부정점수 평균을 구합니다. 

        # 논리구조를 통해 게시글의 최종 감정을 판정합니다. 
        # 긍정/중립/부정 점수를 비교해서 가장 높은 점수의 감정을 게시글의 감정으로 판정합니다. 
        if pos > neg:
            if pos > neut:
                sentiment = "positive"
            else:
                sentiment = "neutral"
        elif pos < neg:
            if neg > neut:
                sentiment = "negative"
            else:
                sentiment = "neutral"
        else:
            if pos > neut:
                sentiment = "positive and negative"
            else:
                sentiment = "neutral"   
                
        return pos, neut, neg, sentiment
    else: 
        return [None, None, None, None]

In [82]:
# 앞서 정의한 토큰화, 감정 분석 함수를 한 번에 실행하는 함수입니다.

def execute_sentiment_analysis(data, lexicon):

  logging("주어진 텍스트에 대해 감정 분석을 시작합니다.")

  for i in tqdm(range(0, len(data))):
    token_list = get_token(data["evaluation"][i])
    data["pos"][i], data["neut"][i], data["neg"][i], data["sentiment"][i] = get_sentiment(token_list)
    data["tokens"][i] = token_list

  
  #csv 파일로 저장하기
  file_name = "/content/drive/My Drive/colab notebooks/textmining/result/textmining_analysis" + ".csv"
  data.to_csv(file_name, index = False, encoding = "utf-8-sig")
  logging("텍스트 감정 분석이 완료되었습니다. 결과를 csv 파일로 저장했습니다.")

  return data

In [84]:
sentiment_result = execute_sentiment_analysis(blog_data, lexicon)
logging("결과 일부를 출력합니다")
sentiment_result[["tokens","pos","neut","neg","sentiment"]].head(10)

| 2020-11-10 | 22:46:07 | 주어진 텍스트에 대해 감정 분석을 시작합니다.



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  import sys


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.



| 2020-11-10 | 22:46:12 | 텍스트 감정 분석이 완료되었습니다. 결과를 csv 파일로 저장했습니다.

| 2020-11-10 | 22:46:12 | 결과 일부를 출력합니다



,tokens,pos,neut,neg,sentiment
0,"[프랑스, 문화, 연구, 과목, 처음, 봐서, 기존, 대면, 어떤, 방식, 했는지는...",0.628501,0.0622566,0.279012,positive
1,"[온라인, 들었던, 다시, 들을, 있다는, 좋았다, 아무래도, 현장, 몰입, 편이,...",0.419899,0.120597,0.352752,positive
2,"[과제, 없이, 진행, 점도, 도움, 되었던, 학습, 평가, 자체, 시험, 충분하게...",0.536883,0.0815146,0.308025,positive
3,"[전체, 흥미, 유익했다, 중간, 중간, 지금, 배우는, 실생활, 어떻게, 응용, ...",0.49704,0.103796,0.275107,positive
4,"[온라인, 퀴즈, 과제, 실시, 했기에, 기말고사, 과목, 다르게, 중간, 기말고사...",0.448135,0.0541859,0.397263,positive
5,"[온라인, 진행, 되었고, 다양한, 방식, 온라인, 한계, 보완, 하면서, 효과, ...",0.547683,0.0551894,0.274208,positive
6,"[사실, 통계, 가장, 자신, 과목, 더욱, 열심히, 따라가려고, 노력, 엄청, 어...",0.487858,0.0736475,0.333855,positive
7,"[좀, 쉽게, 설명, 이해, 돕기, 설명, 개인, 이야기, 덧붙이실, 있었는데, 덕...",0.480213,0.0913536,0.336881,positive
8,"[웹, 프로그래밍, 기초, 웹, 프로그래밍, 응용, 작년, 들었었다, 둘, 성적, ...",0.541814,0.0542764,0.305955,positive
9,"[얼마, 숙명, 포털, 시험, 결과, 확인, 목표, 했던, 성적, 받았다는, 사실,...",0.397848,0.0525544,0.458141,negative


In [86]:
# 앞서 정의한 토큰화, 감정 분석 함수를 한 번에 실행하는 함수입니다.

def execute_sentiment_analysis2(data, lexicon):

  logging("주어진 텍스트에 대해 감정 분석을 시작합니다.")

  for i in tqdm(range(0, len(data))):
    token_list = get_token(data["effort"][i])
    data["pos"][i], data["neut"][i], data["neg"][i], data["sentiment"][i] = get_sentiment(token_list)
    data["tokens"][i] = token_list

  
  #csv 파일로 저장하기
  file_name = "/content/drive/My Drive/colab notebooks/textmining/result/textmining_analysis" + ".csv"
  data.to_csv(file_name, index = False, encoding = "utf-8-sig")
  logging("텍스트 감정 분석이 완료되었습니다. 결과를 csv 파일로 저장했습니다.")

  return data

In [87]:
sentiment_result = execute_sentiment_analysis2(blog_data, lexicon)
logging("결과 일부를 출력합니다")
sentiment_result[["tokens","pos","neut","neg","sentiment"]].head(10)

| 2020-11-10 | 22:48:28 | 주어진 텍스트에 대해 감정 분석을 시작합니다.



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  import sys


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.



| 2020-11-10 | 22:48:30 | 텍스트 감정 분석이 완료되었습니다. 결과를 csv 파일로 저장했습니다.

| 2020-11-10 | 22:48:30 | 결과 일부를 출력합니다



,tokens,pos,neut,neg,sentiment
0,"[매, 예습, 배울, 인지, 먼저, 파악, 했고, 끝난, 이해, 하려고, 노력, 이...",0.480861,0.137363,0.366392,positive
1,"[제때, 제때, 듣고, 자료, 필기, 관해, 궁금한, 생기, 질문, 헷갈리거나, 여...",0.439394,0.272727,0.287879,positive
2,"[도움, 되었던, 방법, 직접, 자료, 프린트, 필기, 방식, 이었는데, 단순히, ...",0.364099,0.0393719,0.471393,negative
3,"[시간, 집중, 들으며, 이해, 최대한, 필기, 남겨, 두려, 하였다, 학기, 온라...",0.349676,0.127181,0.436308,negative
4,[],None,None,None,None
5,[],None,None,None,None
6,"[항상, 미리, 수강, 하면서, 모르는, 빨리, 깨우치기, 노력, 이해, 되지, 않...",0.4778,0.0972036,0.366086,positive
7,"[작가, 이러한, 단서, 독자, 주지, 이런, 세세한, 부분, 설명, 필요하기에, ...",0.422369,0.0763215,0.39951,positive
8,"[들으면서, 실습, 시간, 안, 실습, 문제, 해결, 실습, 문제, 대략, 파악, ...",0.408214,0.0546429,0.482976,negative
9,"[직접, 강의실, 가지, 않고, 집, 들을, 있다는, 편리했지만, 현장, 정해진, ...",0.460791,0.0470085,0.420085,positive
